In [ ]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.utils import multi_runs

import abc
from bokeh.palettes import Category20_9 as cols
import os
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import future.utils
import six

In [61]:
if os.environ.get("DISPLAY", "") == "":
    print("no display found. Using non-interactive Agg backend")
    mpl.use("Agg")


__author__ = "Giulio Rossetti"
__license__ = "BSD-2-Clause"
__email__ = "giulio.rossetti@gmail.com"


@six.add_metaclass(abc.ABCMeta)
class DiffusionPlot(object):
    # __metaclass__ = abc.ABCMeta

    def __init__(self, model, trends):
        self.model = model
        self.trends = trends
        statuses = model.available_statuses
        self.srev = {v: k for k, v in future.utils.iteritems(statuses)}
        self.ylabel = ""
        self.title = ""
        self.nnodes = model.graph.number_of_nodes()
        self.normalized = True

    @abc.abstractmethod
    def iteration_series(self, percentile):
        """
        Prepare the data to be visualized

        :param percentile: The percentile for the trend variance area
        :return: a dictionary where iteration ids are keys and the associated values are the computed measures
        """
        pass

    def plot(self, filename=None, percentile=90, statuses=None, descr=None):
        """
        Generates the plot

        :param filename: Output filename
        :param percentile: The percentile for the trend variance area
        :param statuses: List of statuses to plot. If not specified all statuses trends will be shown.
        """

        pres = self.iteration_series(percentile)
        # infos = self.model.get_info()
        # descr = ""

        plt.figure(figsize=(20, 15))

        # for k, v in future.utils.iteritems(infos):
        #     descr += "%s: %s, " % (k, v)
        # descr = descr[:-2].replace("_", " ")

        mx = 0
        i = 0
        for k, l in future.utils.iteritems(pres):

            if statuses is not None and self.srev[k] not in statuses:
                continue
            mx = len(l[0])
            if self.normalized:
                plt.plot(
                    range(0, mx),
                    l[1] / self.nnodes,
                    lw=2,
                    label=self.srev[k],
                    alpha=0.5,
                )  # , color=cols[i])
                plt.fill_between(
                    range(0, mx), l[0] / self.nnodes, l[2] / self.nnodes, alpha=0.2
                )
                # ,color=cols[i])
            else:
                plt.plot(
                    range(0, mx), l[1], lw=2, label=self.srev[k], alpha=0.5
                )  # , color=cols[i])
                plt.fill_between(range(0, mx), l[0], l[2], alpha=0.2)  # ,color=cols[i])

            i += 1

        plt.grid(axis="y")
        # plt.title(descr)
        plt.xlabel("Iterations", fontsize=26)
        plt.ylabel(self.ylabel, fontsize=26)
        plt.tick_params(axis='both', which='major', labelsize=20)
        plt.legend(loc="best", fontsize=24)
        plt.title(f"fraction infected: {descr[0]}, beta: {descr[1]}, gamma: {descr[2]}", fontsize=26) 
        plt.xlim((0, mx))

        plt.tight_layout()
        if filename is not None:
            plt.savefig(filename)
            plt.clf()
        else:
            plt.show()

class DiffusionTrend(DiffusionPlot):
    def __init__(self, model, trends):
        """
        :param model: The model object
        :param trends: The computed simulation trends
        """
        super(self.__class__, self).__init__(model, trends)
        self.ylabel = "#Nodes"
        self.title = "Diffusion Trend"

    def iteration_series(self, percentile):

        series = {k: [] for k in self.srev.keys()}

        presences = {k: [] for k in self.srev.keys()}
        for t in self.trends:

            for st in t:
                for k in t[st]["node_count"]:
                    presences[k].append(np.array(t[st]["node_count"][k]))

        for st in presences:
            tp = np.percentile(np.array(presences[st]), percentile, axis=0)
            bp = np.percentile(np.array(presences[st]), 100 - percentile, axis=0)
            av = np.average(np.array(presences[st]), axis=0)
            series[st] = (tp, av, bp)

        return series

no display found. Using non-interactive Agg backend


In [ ]:
class Graph:
    def __init__(self, num_of_nodes):
        self.num_of_nodes = num_of_nodes

    def barabasi_albert(self, num_of_edges):
        return nx.barabasi_albert_graph(self.num_of_nodes, num_of_edges)

    def watts_strogatz(self, k_nearest_neighbors, rewiring_probability):
        return nx.watts_strogatz_graph(self.num_of_nodes, k_nearest_neighbors, rewiring_probability)

    def erdos_renyi(self, edge_creation_probability):
        return nx.erdos_renyi_graph(self.num_of_nodes, edge_creation_probability)


def set_config(beta, gamma, fraction_infected):
    config = mc.Configuration()
    config.add_model_parameter('beta', beta)
    config.add_model_parameter('gamma', gamma)
    config.add_model_parameter("fraction_infected", fraction_infected)
    return config

def run_simulation(beta, gamma, fraction_infected, graph, network_abbrv):        
    config = set_config(beta, gamma, fraction_infected)
    sir_model = ep.SIRModel(graph)
    sir_model.set_initial_status(config)
    trends = multi_runs(sir_model, execution_number=10, iteration_number=100, infection_sets=None, nprocesses=4)
    viz = DiffusionTrend(sir_model, trends)
    viz.plot(filename=f"../assets/{network_abbrv}_trend_beta{beta}_gamma{gamma}_infected{fraction_infected}.png", descr=[fraction_infected, beta, gamma])

# Set network model params
graph = Graph(num_of_nodes=1000)
ba_graph = graph.barabasi_albert(num_of_edges=5)
ws_graph = graph.watts_strogatz(k_nearest_neighbors=6, rewiring_probability=0.1)
er_graph = graph.erdos_renyi(edge_creation_probability=0.1)

In [62]:
# Set disease params
infection_rate = 0.1
recovery_rate = 0.05
fraction = 0.01

# Usage
run_simulation(infection_rate, recovery_rate, fraction, ba_graph, 'BA')
run_simulation(infection_rate, recovery_rate, fraction, ws_graph, 'WS')
run_simulation(infection_rate, recovery_rate, fraction, er_graph, 'ER')